<a href="https://colab.research.google.com/github/Lucasdmnasc/JogoDeGatinho/blob/main/JogoGatinho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

"""
    Estrutura de Dados - SME0827
    Grupo 4:
        Lucas Della Montagna (11218278)
        Mateus Souza Pires (11833639)
        Vivian Kiyomi Amano  (11300502)
"""

import os
import sys
from random import choice
import pandas as pd
import re
import string
from copy import deepcopy
from datetime import datetime as dt
import time
from collections import namedtuple


# cria sting do ambiente
template = """ _______________________________________________________________________________
|                                                                               |
|                                                                               |
|                              ^^^^^^^^^^^^^^^^^^^^                             |
|                                                                               |
|_______________________________________________________________________________|
|           |                           |                           |           |
|     <     |188888888888888888888888888|288888888888888888888888888|     >     |
|     <     |177777777777777777777777777|277777777777777777777777777|     >     |
|     <     |166666666666666666666666666|266666666666666666666666666|     >     |
|     <     |155555555555555555555555555|255555555555555555555555555|     >     |
|     <     |144444444444444444444444444|244444444444444444444444444|     >     |
|     <     |133333333333333333333333333|233333333333333333333333333|     >     |
|     <     |122222222222222222222222222|222222222222222222222222222|     >     |
|     <     |111111111111111111111111111|211111111111111111111111111|     >     |
|     <     |             1             |             2             |     >     |
|     <     |___________________________|___________________________|     >     |
|     <     |                           |                           |     >     |
|     <     |388888888888888888888888888|488888888888888888888888888|     >     |
|     <     |377777777777777777777777777|477777777777777777777777777|     >     |
|     <     |366666666666666666666666666|466666666666666666666666666|     >     |
|     <     |355555555555555555555555555|455555555555555555555555555|     >     |
|     <     |344444444444444444444444444|444444444444444444444444444|     >     |
|     <     |333333333333333333333333333|433333333333333333333333333|     >     |
|     <     |322222222222222222222222222|422222222222222222222222222|     >     |
|     <     |311111111111111111111111111|411111111111111111111111111|     >     |
|     <     |             3             |             4             |     >     |
|___________|___________________________|___________________________|___________|
|                                                                               |
|                                     *YOU*                                     |
|_______________________________________________________________________________|"""


# ambiente

class EnvManager:

    # gerenciador de ambientes

    def __init__(self, ambientes=[], next_ones=[]):
        self.ambientes = ambientes
        self.next_ones = next_ones

    def __str__(self):
        # representação dos ambientes como string
        return " -> ".join(self.ambientes)

    def size(self):
        # retorna quantidade de ambientes
        return len(self.ambientes)

    def ambiente_atual(self):
        # retorna o objeto referente ao ambiente atual
        return self.ambientes[-1]

    def next(self):
        # tenta avançar para o próximo ambiente
        if len(self.next_ones):
            self.ambientes += [deepcopy(self.next_ones[0])]
            self.next_ones = deepcopy(self.next_ones[1:])
        else:
            raise Exception("Não foi possível avançar pois você já está no último ambiente")

    def previous(self):
        # tenta retornar para o ambiente anterior
        if len(self.ambientes) == 1:
            raise Exception("Não foi possível voltar pois você já está no primeiro ambiente")
            
        else:
            self.next_ones = [deepcopy(self.ambientes[-1])] + self.next_ones
            del self.ambientes[-1]


class Objeto:

    # classe que encapsula objetos genéricos para serem armazenados em uma pilha ou uma fila

    def __init__(self, nome):
        # armazena o objeto (nome) como um atributo da classe
        self.nome = nome.capitalize()
        self.próximo = None


    def __str__(self):
        # Método para representar o objeto como uma string
        return self.nome

    def __add__(self, other):
        return (str(self), str(other))

    def __radd__(self, other):
        return (str(other), str(self))


class MetroQuadrado:

    # pilha que abststrai um metro quadrado (m^2) da superfície do chão

    def __init__(self, stack_max_size):

        # instancia um objeto 'vazio' e o armazena como um atributo da classe
        self.elemento_do_topo = Objeto("Vazio")

        # incia contador de objetos nesse m^2
        self.quantidade_de_objetos = 0
        self.stack_max_size = stack_max_size

    def quantidade_de_objetos_no_metro_quadrado(self):
        # retorna a quantidade de objetos nesse metro quadrado
        return self.quantidade_de_objetos
    

    def metro_quadrado_esta_vazio(self):
        # retorna True se esse metro quadrado contém algum objeto, caso contrário False
        if self.quantidade_de_objetos == 0:
            return True
        return False
    

    def elemento_do_topo_da_pilha(self):
        # retorna o elemento do topo da pilha
        if self.metro_quadrado_esta_vazio():
            raise Exception("Esse metro quadrado não contém nenhum objeto")
        return self.elemento_do_topo.próximo.nome


    def empilha_objeto(self, objeto):
        """
            Parametros:

                objeto (TAD): objeto a ser empilhado nesse metro quadrado
        """
        if self.quantidade_de_objetos < self.stack_max_size:

            # insere o elemento dado no topo da pilha desse metro quadrado
            objeto.próximo = self.elemento_do_topo.próximo
            self.elemento_do_topo.próximo = objeto
            self.quantidade_de_objetos += 1
            return True
        return False
    
    def desempilha_objeto(self):
        # remove o elemento do topo da pilha desse metro quadrado e o retorna
        if self.metro_quadrado_esta_vazio():
            raise Exception("Esse metro quadrado não contém nenhum objeto, não foi possível desempilhar")
        removed = self.elemento_do_topo.próximo
        self.elemento_do_topo.próximo = self.elemento_do_topo.próximo.próximo
        self.quantidade_de_objetos -= 1
        return removed


    def lê_todos_objetos_da_pilha(self):
        # lê todos os objetos empilhados nesse metro quadrado
        próximo = self.elemento_do_topo
        objetos_empilhados = [próximo.nome]
        while próximo.próximo:
            objetos_empilhados += [próximo.próximo.nome]
            próximo = próximo.próximo
        while "Vazio" in objetos_empilhados:
            objetos_empilhados.remove('Vazio')
        return reversed(objetos_empilhados)       


# ambiente

class Chão:

    # TAD que abstrai o chão do ambiente
    # O 'chão' é um conjunto de pilhas (metros quadrados)
    # Quando um objeto é lançado, ele pode cair em qualquer lugar
    # Se muitos objetos são lançados, eles podem ser 'empilhados'

    def __init__(self, dimensão_do_ambiente=4, stack_max_size=8):
        """
            Parametros:

                dimensão_do_ambiente (int): metros quadrados do ambiente
        """
        # dimensão do ambiente (em metros quadrados)
        self.dimensão_do_ambiente = dimensão_do_ambiente
        self.stack_max_size = stack_max_size

        # inicia índice de metros quadrados
        self.metros_quadrados = {}

        # particiona o ambiente em metros quadrados para serem tratados como pilhas diferentes
        for i in range(dimensão_do_ambiente):

            # cria e armazena o metro quadrado em um atributo da classe
            self.metros_quadrados[i+1] = MetroQuadrado(stack_max_size=self.stack_max_size)


    def adiciona_objeto_ao_metro_quadrado(self, objeto, índice_do_metro_quadrado):
        """
            Método que adiciona um dado objeto no dado metro quadrado

            Parâmetros:

                objeto (TAD): objeto a ser adicionado
                índice_do_metro_quadrado (int): índice referente ao metro quadrado onde o objeto será armazenado

            Retorna:

                boolean: se o objeto foi adicionado com sucesso
        """
        # checa se o lugar onde o objeto será armazenado é válido
        if índice_do_metro_quadrado not in self.metros_quadrados.keys():
            return False

        # armazena a mobília
        self.metros_quadrados[índice_do_metro_quadrado].empilha_objeto(objeto)


        return True

    def get_pilhas_chão(self):
        # retorna índice do m^2 e lista de objetos dispostos nas pilhas do chão
        output_lst = []
        index = list(self.metros_quadrados.keys())
        for índice, metro_quadrado in self.metros_quadrados.items():
            output_lst += [metro_quadrado.lê_todos_objetos_da_pilha()]
        return index, output_lst


class Ambiente:
    
    # TAD que abstrai um ambiente

    def __init__(self, nome, dimensão_do_ambiente, stack_max_size, template):
        self.nome = nome.capitalize()
        self.lugares = ['direita', 'esquerda', 'frente', 'chão']
        self.direita = None 
        self.esquerda = None
        self.frente = None
        self.móveis = []
        self.template = template

        if dimensão_do_ambiente < 2:
            raise Exception("Ambiente precisa ter mais do que 2 metros quadrados")

        self.dimensão_do_ambiente = dimensão_do_ambiente
        self.chão = Chão(dimensão_do_ambiente=self.dimensão_do_ambiente, stack_max_size=stack_max_size)


    def adiciona_mobília(self, mobília, lugar='esquerda'):
        """
            Método que adiciona uma mobília (TAD) à um lugar no ambiente
            
            Parâmetros:

                mobília (TAD): mobília para adicionar
                lugar (string): onde a mobília será armazenada

            Retorna:

                boolean: se a mobília foi adicionada com sucesso
        """

        # checa se o lugar onde a mobília será armazenada é válido
        if lugar != 'chão' and lugar not in self.lugares:
            return False

        # armazena a mobília
        setattr(self, lugar, mobília)
        self.móveis += [mobília]

        return getattr(self, lugar) == mobília


    def adiciona_embutidos(self, mobília, lugar='esquerda'):
        """
            Método que adiciona uma mobília (TAD) à um lugar no ambiente
            
            Parâmetros:

                mobília (TAD): mobília para adicionar
                lugar (string): onde a mobília será armazenada

            Retorna:

                boolean: se a mobília foi adicionada com sucesso
        """

        # checa se o lugar onde a mobília será armazenada é válido
        if lugar != 'chão' and lugar not in self.lugares:
            return False

        # armazena a mobília
        setattr(self, lugar, mobília)

        return getattr(self, lugar) == mobília


    def joga_objeto_no_chão(self, objeto, ignore=[]):
        """
            Método que adiciona objeto dado à uma dos metros quadrados (pilhas) [aleatoriamente]

            Parâmetros:

                objeto (TAD): objeto a ser adicionado

            Retorna:

                boolean: se o objeto foi adicionado com sucesso
        """
        while 1:
            # escolhe aleatoriamente um índice referente à um metro quadrado que receberá o objeto
            índice_do_metro_quadrado = choice([i for i in range(1, self.dimensão_do_ambiente) if i not in ignore])
            success = self.chão.adiciona_objeto_ao_metro_quadrado(objeto, índice_do_metro_quadrado)
            if success: # se foi possível adicionar objeto 
                break
        return success


    def __str__(self):
        
        print("\nVocê está na %s" % self.nome)

        # obter todos os móveis/lugares dentro do ambiente atual        
        ambiente = {}
        for lugar in self.lugares:
            ambiente[lugar] = getattr(self, lugar)

        def replace(template_str, placeholder, word):
            # Inner function to replace placeholders in string
            letters = list(word)
            pattern = placeholder if placeholder in placeholder in string.ascii_letters else "\\%s" % placeholder
            for i in re.finditer(pattern, template_str):

                start, end = i.span()
                template_str = template_str[:start] + letters[0] + template_str[end:]
                del letters[0]

            return template_str

        # inserir móveis no template
        length = self.template.count('^')
        template = replace(self.template, '^', ambiente['frente'].nome.center(length, ' '))
        template = replace(template, '<', ambiente['esquerda'].nome.center(length, ' '))
        template = replace(template, '>', ambiente['direita'].nome.center(length, ' '))
        
        index, table = ambiente['chão'].get_pilhas_chão()

        # formata impressão de metros quadrados (pilhas)
        table = pd.DataFrame(table, index=index)
        for _ in range(ambiente['chão'].stack_max_size - len(table.T)): # fill all stacks based on stack's max len
            table = table.T.append(pd.Series(dtype='object'), ignore_index=True).T
        table.fillna('', inplace=True)

        stacks = {}
        for idx, row in table.iterrows():
            stacks[idx] = list(row.values)


        # replace ground stacks
        for m2_idx, m2_stack in stacks.items():
            for obj_idx, obj in enumerate(m2_stack, 1):
                placeholder = str(m2_idx) + (str(obj_idx) * 26)
                
                template = template.replace(placeholder, obj.center(27, ' '))

        return template


# inventário do jogador

class Inventário:

    # fila que abstrai um inventário onde o personagem pode guardar uma quantidade finita de objetos

    def __init__(self, quantidade_máxima_de_objetos):
        self.quantidade_máxima_de_objetos = quantidade_máxima_de_objetos
        self.inventário = []

    def __str__(self):
        # Monta string para impressão do inventário
        if len(self.inventário):
            saída = "\nO seu inventário contém os seguintes items:"
            for item in self.inventário:
                saída += '\n' + item.__str__()
            return saída + '\n'
        else:
            return "\nO seu inventário está vazio"

    
    def adiciona_objeto_ao_inventário(self, objeto):
        # Insere um novo elemento no fim da fila
        if len(self.inventário) == self.quantidade_máxima_de_objetos:
            first = self.inventário[0]
            print("O inventário está cheio. Descartando o objeto %s para adicionar o %s" % (first, objeto))
            self.inventário = self.inventário[1:] + [objeto]
            return first
        else:
            self.inventário = self.inventário + [objeto]
            return None


    def remove_objeto_do_inventário(self, nome_do_objeto):
        # Remove o objeto do inventário e o retorna
        if not len(self.inventário):
            print("o seu inventário já está vazio. Não foi possível remover o elemento")
            return False
        objeto_a_remover = [i for i in self.inventário if i.nome == nome_do_objeto]
        if not objeto_a_remover:
            print("Não existe nenhum %s no inventário" % nome_do_objeto)
            return False
        self.inventário = [i for i in self.inventário if i not in objeto_a_remover]
        return objeto_a_remover[0]


# objeto interativo genérico

class ObjetoInterativo:

    def __init__(self, nome, mensagem):
        self.nome = nome
        self.mensagem = mensagem

    def __str__(self):
        return self.nome


## móveis:

class TV:
    
    def __init__(self, nome, mensagem, filmes):
        self.nome = nome.capitalize()
        self.mensagem = mensagem
        self.filmes = filmes
        self.filme = self.filmes[0].nome
        self.descrição = self.filmes[0].descrição
      
    def watch(self):
        # comano interativo
        print("%s: %s" % (self.filme, self.descrição))
      
    def ver_programação(self):
        # comano interativo
        print("A %s está ligada. O filme que está passando %s" % (self.nome, self.filme)) 
        
    def help(self):
        # comano interativo
        print("Você pode assitir o filme através do comando 'watch'")
        print("Você pode conferir filme que está sendo exibido através do comando 'ver_programação'")
        print("Você pode mudar de filme através do comando 'change'")
        print("Você pode parar de assistir através do comando 'stop'")

    def listar_filmes(self):
        # comano interativo
        for idx, filme in enumerate(self.filmes):
            print("Filme %s: %s" % (idx, filme.nome), "-> Descrição: %s" % filme.descrição)
            print()
      
    def change(self):
        # comano interativo
        self.listar_filmes()
        idx_novo_filme = int(input("Qual o índice do filme deseja assistir? "))
        self.filme = self.filmes[idx_novo_filme].nome
        self.descrição = self.filmes[idx_novo_filme].descrição
        
    def __str__(self):
        return self.nome


class Sofá:

    def __init__(self, nome, mensagem):
        self.nome = nome.capitalize()
        self.mensagem = mensagem

    def help(self):
        # comano interativo
        print("Você pode tirar uma soneca através do comando 'soneca'")
        print("Você pode parar de interagir com o sofá através do comando 'stop'")

    def soneca(self):
        # comano interativo
        print("Agora você está descansado e pronto para prosseguir")

    def __str__(self):
        return self.nome


class Armário:

    def __init__(self, nome, mensagem, objetos_sobre_armário, ambiente):
        self.nome = nome.capitalize()
        self.mensagem = mensagem
        self.objetos_sobre_armário = objetos_sobre_armário
        self.ambiente = ambiente

    def help(self):
        # comano interativo
        print("Você pode utilizar suas habilidades felinas e derrubar os objetos que estão sobre o armário através do comando 'cat_superpower'")
        print("Você pode parar de interagir com o armário através do comando 'stop'")

    def cat_superpower(self):
        # comano interativo
        for objeto in self.objetos_sobre_armário:
            self.ambiente.joga_objeto_no_chão(objeto)

    def __str__(self):
        return self.nome


class Mesa:

    def __init__(self, nome, mensagem, objetos_sobre_mesa, ambiente):
        self.nome = nome.capitalize()
        self.mensagem = mensagem
        self.objetos_sobre_mesa = objetos_sobre_mesa
        self.ambiente = ambiente

    def help(self):
        # comano interativo
        print("Você pode utilizar suas habilidades felinas e derrubar os objetos que estão sobre a mesa através do comando 'cat_superpower'")
        print("Você pode parar de interagir com a mesa através do comando 'stop'")

    def cat_superpower(self):
        # comano interativo
        for objeto in self.objetos_sobre_mesa:
            self.ambiente.joga_objeto_no_chão(objeto)

    def __str__(self):
        return self.nome


class Cama:

    def __init__(self, nome, mensagem):
        self.nome = nome.capitalize()
        self.mensagem = mensagem

    def help(self):
        # comano interativo
        print("Você pode tirar uma soneca através do comando 'soneca'")
        print("Você pode parar de interagir com a cama através do comando 'stop'")

    def soneca(self):
        # comano interativo
        print("Agora você está descansado e pronto para prosseguir")

    def __str__(self):
        return self.nome


class EstanteGrande:

    def __init__(self, nome, mensagem):
        self.nome = nome.capitalize()
        self.mensagem = mensagem

    def help(self):
        # comano interativo
        print("Parece que um gato passou por aqui e usou seu superpoderes para derrubar todos os livros da estante grande")
        print("Você pode parar de interagir com a estante grande através do comando 'stop'")

    def __str__(self):
        return self.nome


class EstantePequena:

    def __init__(self, nome, mensagem):
        self.nome = nome.capitalize()
        self.mensagem = mensagem

    def help(self):
        # comano interativo
        print("Parece que um gato passou por aqui e usou seu superpoderes para derrubar todos os livros da estante pequena")
        print("Você pode parar de interagir com a estante grande através do comando 'stop'")

    def __str__(self):
        return self.nome


class Janela:

    def __init__(self, nome, mensagem):
        self.nome = nome.capitalize()
        self.mensagem = mensagem
        self.allow_break = False

    def look(self):
        # comano interativo
        print()
        print(self.mensagem)

    def help(self):
        # comano interativo
        print("Você pode olhar pela janela através do comando 'look'")
        print("Você pode parar de interagir com a %s através do comando 'stop'" % self.nome)
        print("Você pode quebrar a janela e finalizar o jogo através do comando 'break_window'")

    def break_window(self):
        # comano interativo
        if self.allow_break:
            print("\nCRASH!")
            time.sleep(4)
            return "CRASH!"
        else:
            print("\nVocê precisa de uma pedra no inventário para poder quebrar a janela...")
            time.sleep(4)
            return None

    def __str__(self):
        return self.nome


# embutidos

class Porta:

    # TAD que abstrai uma porta para transição de ambientes

    def __init__(self, antes, depois):
        self.antes = antes
        self.depois = depois
        self.nome = "Porta p/ %s" % self.depois.nome

    def __str__(self):
        return self.nome

    def __str__(self):
        return self.nome



if __name__ == "__main__":

    print("\nVocê é um felino e está com fome.")
    print("Você assistiu o filme 'Os sem florestas' e ficou inspirado a invadir a casa de humanos para obter alimento.")
    print("Seu amigo gato Frajola pensou o mesmo e decidiu ir primeiro.")
    print("Frajola é muito desajeitado e acabou fazendo uma bagunça enorme na casa invadida, derrubando tudo no chão.")
    print("Como Frajola não conseguiu trazer alimento, ele pediu sua ajuda.")
    print("Vários objetos e alimentos estão no chão, empilhados.")
    print("Suas patinhas não te permitem ter contato com itens que não estão no topo da pilha.")
    print("Ao remover um item da pilha, ele provavelmente cairá sobre outos itens, dificultando a sua tarefa.")
    print("Agora é com você. Entre na casa, descubra o mundo humano e colete o máximo de alimentos no menor tempo possível.\n")


    # cria inventário e adicionando objetos iniciais
    inventário = Inventário(quantidade_máxima_de_objetos=4)


    # cria cenário
    sala = Ambiente(nome='sala', dimensão_do_ambiente=4, stack_max_size=8, template=template)
    cozinha = Ambiente(nome='cozinha', dimensão_do_ambiente=4, stack_max_size=8, template=template)
    quarto = Ambiente(nome='quarto', dimensão_do_ambiente=4, stack_max_size=8, template=template)
    biblioteca = Ambiente(nome='biblioteca', dimensão_do_ambiente=4, stack_max_size=8, template=template)


    # cria portas
    porta1 = Porta(antes=sala, depois=cozinha)
    porta2 = Porta(antes=cozinha, depois=quarto)
    porta3 = Porta(antes=quarto, depois=biblioteca)

    # adiciona portas aos ambientes
    sala.adiciona_embutidos(mobília=porta1, lugar='frente')
    cozinha.adiciona_embutidos(mobília=porta2, lugar='frente')
    quarto.adiciona_embutidos(mobília=porta3, lugar='direita')


    ## cria mobília e as adiciona ao ambiente
    Filme = namedtuple('Filme', 'nome descrição')
    tv = TV(nome='TV 72 polegadas', mensagem="Adoro filmes!", filmes=[
        Filme('Titanic', 'Esse filme te fará chorar'),
        Filme('Crepusculo', 'Esse filme tbm te fará chorar')
    ])
    sofá = Sofá(nome='sofá', mensagem='Que soninho!')
    sala.adiciona_mobília(mobília=sofá, lugar='esquerda')
    sala.adiciona_mobília(mobília=tv, lugar='direita')


    óculos = ObjetoInterativo("Óculos", mensagem="Óculos de leitura... Parece que temos um leitor aqui!")
    faca_de_churrasco = ObjetoInterativo("Faca de churrasco", mensagem="Que perigo!")
    armário = Armário(nome='armário', mensagem='Tenho que tomar cuidado para não derrubar tudo :)', objetos_sobre_armário=[óculos], ambiente=cozinha)
    mesa = Mesa(nome='mesa', mensagem='Tenho que tomar cuidado para não derrubar tudo :)', objetos_sobre_mesa=[faca_de_churrasco], ambiente=cozinha)
    cozinha.adiciona_mobília(mobília=armário, lugar='esquerda')
    cozinha.adiciona_mobília(mobília=mesa, lugar='direita')


    cama = Cama('cama', 'Que soninho!')
    Filme = namedtuple('Filme', 'nome descrição')
    tv2 = TV(nome='TV 50 polegadas', mensagem="Adoro filmes!", filmes=[
        Filme('As branquelas', 'Esse filme te fará rir'),
        Filme('Garfield', 'Esse filme tbm te fará rir')
    ])
    quarto.adiciona_mobília(mobília=cama, lugar='frente')
    quarto.adiciona_mobília(mobília=tv2, lugar='esquerda')


    estante_grande = EstanteGrande(nome='estante grande', mensagem='Parece que um gato passou por aqui e usou seu superpoderes para derrubar todos os livros da estante grande')
    estante_pequena = EstantePequena(nome='estante pequena', mensagem='Parece que um gato passou por aqui e usou seu superpoderes para derrubar todos os livros da estante pequena')
    janela = Janela(nome='janela', mensagem='O céu está bonito lá fora')
    biblioteca.adiciona_mobília(mobília=estante_pequena, lugar='direita')
    biblioteca.adiciona_mobília(mobília=estante_grande, lugar='frente')
    biblioteca.adiciona_mobília(mobília=janela, lugar='esquerda')


    # criação de objetos interativos
    
    ##sala##
    molho_de_chaves = ObjetoInterativo(nome='Molho de chaves', mensagem='Chaves do carro, da casa e da janela, pena que não sei usá-las')
    sala.joga_objeto_no_chão(molho_de_chaves)
    
    relógio = ObjetoInterativo(nome='Relógio', mensagem='Já são seis horas da manhã?')
    sala.joga_objeto_no_chão(relógio)
    
    caneta_bic = ObjetoInterativo(nome='Caneta bic', mensagem='Minhas patinhas não me permitem coletar isto')
    sala.joga_objeto_no_chão(caneta_bic)
    
    controle_remoto = ObjetoInterativo(nome='Controle remoto', mensagem='Não tenho tempo para assistir TV agora')
    sala.joga_objeto_no_chão(controle_remoto)
    
    roteador_do_wifi = ObjetoInterativo(nome='Roteador wifi', mensagem='Acho que os humanos vão ficar muito bravos se eu coletar isso')
    sala.joga_objeto_no_chão(roteador_do_wifi)
    
    pedra = ObjetoInterativo(nome='Pedra', mensagem='Isso pode ser muito útil para quebrar uma janela')
    sala.joga_objeto_no_chão(pedra)

    ração = ObjetoInterativo(nome='Ração', mensagem='Hmmm, de frango com carne')
    sala.joga_objeto_no_chão(ração)
    
    ratinho_de_borracha = ObjetoInterativo(nome='Ratinho de borracha', mensagem='Ótima distração, não vai funcionar comigo humanoss!')
    sala.joga_objeto_no_chão(ratinho_de_borracha)

    osso_para_cachorros = ObjetoInterativo(nome='Osso para cachorros', mensagem='Como eles conseguem roer isso?')
    sala.joga_objeto_no_chão(osso_para_cachorros)



    ##biblioteca##
    dom_casmurro = ObjetoInterativo(nome='Livro Dom Casmurro', mensagem='Capitú, traiu ou não traiu?')
    biblioteca.joga_objeto_no_chão(dom_casmurro)

    pequeno_príncipe = ObjetoInterativo(nome='Livro O Pequeno Príncipe', mensagem='Isso obviamente não é um chapéu')
    biblioteca.joga_objeto_no_chão(pequeno_príncipe)
    
    vidas_secas = ObjetoInterativo(nome='Livro Vidas Secas', mensagem='Por mais que seja um cachorro, a Baleia não merecia esse fim')
    biblioteca.joga_objeto_no_chão(vidas_secas)
    
    o_cortiço = ObjetoInterativo(nome='Livro O Cortiço', mensagem='Eles precisam disso para passar na fuvest??')
    biblioteca.joga_objeto_no_chão(o_cortiço)
    
    livro_1984 = ObjetoInterativo(nome='Livro 1984', mensagem='Imagina se isso vira um reality show?')
    biblioteca.joga_objeto_no_chão(livro_1984)
    
    revolução_dos_bixos = ObjetoInterativo(nome='Livro A Revolução dos Bixos', mensagem='E se fizessemos a revolução dos gatos?')
    biblioteca.joga_objeto_no_chão(revolução_dos_bixos)
    
    diário = ObjetoInterativo(nome='Diário', mensagem=' "NÃO LEIA" ')
    biblioteca.joga_objeto_no_chão(diário)
    
    croquetas = ObjetoInterativo(nome='Croquetas', mensagem='O segundo melhor petisco do mundo!')
    biblioteca.joga_objeto_no_chão(croquetas)
        
    
    
    ##cozinha###
    bolacha = ObjetoInterativo(nome='Bolacha', mensagem='Isso não se chama biscoito?')
    cozinha.joga_objeto_no_chão(bolacha)
    
    maçã = ObjetoInterativo(nome='Maçã', mensagem='Muito saudável pra mim')
    cozinha.joga_objeto_no_chão(maçã)
    
    garfo = ObjetoInterativo(nome='Garfo', mensagem='Tenho que tomar cuidado com esses objetos pontiagudos')
    cozinha.joga_objeto_no_chão(garfo)
    
    prato = ObjetoInterativo(nome='Prato', mensagem='Olha! um potinho de ração gigante')
    cozinha.joga_objeto_no_chão(prato)
    
    buxinha = ObjetoInterativo(nome='Buxinha', mensagem='Muito macia!')
    cozinha.joga_objeto_no_chão(buxinha)
    
    copo = ObjetoInterativo(nome='Copo', mensagem='Aqui que os humanos tomam leite?')
    cozinha.joga_objeto_no_chão(copo)
    
    frigideira = ObjetoInterativo(nome='Frigideira', mensagem='Aço inox antiaderente!')
    cozinha.joga_objeto_no_chão(frigideira)
    
    atum = ObjetoInterativo(nome='Atum', mensagem='Eu preferiria salmão, mas serve')
    cozinha.joga_objeto_no_chão(atum)

    lasanha = ObjetoInterativo(nome='Lasanha', mensagem='Estou ficando com fome e por sorte esse é meu alimento favorito')
    cozinha.joga_objeto_no_chão(lasanha)
    
    pizza = ObjetoInterativo(nome='Pizza', mensagem='Mama mia!')
    cozinha.joga_objeto_no_chão(pizza)

    pescado = ObjetoInterativo(nome='Pescado', mensagem='Nham nham!')
    cozinha.joga_objeto_no_chão(pescado)



    ##quarto##
    whiskas_sachê = ObjetoInterativo(nome='Whiskas sachê', mensagem='Será que humanos também comem isso')
    quarto.joga_objeto_no_chão(whiskas_sachê)
    
    camiseta = ObjetoInterativo(nome='Camiseta', mensagem='Eu também curto Guns and Roses')
    quarto.joga_objeto_no_chão(camiseta)
    
    bola_de_basquete = ObjetoInterativo(nome='Bola de basquete', mensagem='Esse é o quarto do Michal Jordan?')
    quarto.joga_objeto_no_chão(bola_de_basquete)
    
    tênis = ObjetoInterativo(nome='Tênis', mensagem='Apenas 1 par, puts')
    quarto.joga_objeto_no_chão(tênis)
    
    gibi_do_Garfield = ObjetoInterativo(nome='Gibi do Garfield', mensagem='Olha aí, meu ídolo')
    quarto.joga_objeto_no_chão(gibi_do_Garfield)
    
    edredom = ObjetoInterativo(nome='Edredom', mensagem='Eu poderia dormir com isso nesse frio')
    quarto.joga_objeto_no_chão(edredom)
    
    caderno = ObjetoInterativo(nome='Caderno', mensagem='Não Sabia que probabilidade fosse tão díficil')
    quarto.joga_objeto_no_chão(caderno)
    
    celular = ObjetoInterativo(nome='Celular', mensagem='Vish! 32 ligações perdidas da namorada..')
    quarto.joga_objeto_no_chão(celular)
    
    notebook = ObjetoInterativo(nome='Notebook', mensagem='Gamer?')
    quarto.joga_objeto_no_chão(notebook)
        

    # configurações do jogo
    player_name = input("Dê um nome para seu personagem: ") or "Caramelo"    
    print("\nO seu personagem se chama %s" % player_name)

    input("\nPressione enter para iniciar o jogo...")
    game_start = dt.now()

    # estado do jogo
    game_state = {
        'env manager': EnvManager(ambientes=[sala], next_ones=[cozinha, quarto, biblioteca]),
        'inventário': inventário
    }

    win = False
    while 1:

        if win:
            break

        print(game_state['env manager'].ambiente_atual())
        print(game_state['inventário'])

        # TODO: trazer comandos e help para o nível do ambiente. Vai facilitar muito
        
        comandos = {
            'pick': 'Comando utilizado para coletar objeto do chão para o inventário. Sintaxe: pick from 1',
            'drop': 'Comando utilizado para remover objeto do inventário. Sintaxe: drop nome_objeto',
            'unstack': 'Comando utilizado para remover elemento do topo de uma das pilhas. Sintaxe: unstack 1',
            'next_env': 'Comando utilizado para ir para o próximo ambiente, Sintaxe: next_env',
            'prev_env': 'Comando utilizado para voltar para o ambiente anterior, Sintaxe: prev_env',
            'interact': 'Comando utilizado para interagir com a mobília. Sintaxe: interact with sofá'
        }
        
        print("Os comandos permitidos estão listados abaixo:\n")
        for cmd, description in comandos.items():
            print("%s: %s" % (cmd, description))
        print("quit: parar de jogar\n")
        
        cmd = input("\nInsira um comando: ")
        
        # checar se jogador deseja parar de jogar
        if 'quit' in cmd:
            sys.exit()

        if 'help' in cmd:
            for cmd, description in comandos.items():
                print("%s: %s" % (cmd, description))
            continue
        
        # validar comando
        try:
            
            # obtém a ação do comando     
            action = cmd.split()[0]

            # verifica se ação é válida
            if action not in comandos.keys():
                raise Exception("Ação inválida")
          
            if action == 'pick':
                
                # obtém índice da pilha que usuário deseja interagir
                idx_pilha = int(re.search(r'\d', cmd).group(0))
                
                # obtém objeto da pilha desejada
                m2 = game_state['env manager'].ambiente_atual().chão.metros_quadrados[idx_pilha]
                obj = m2.desempilha_objeto()
                
                # adiciona objeto ao inventário
                removed = game_state['inventário'].adiciona_objeto_ao_inventário(obj)
                if removed:
                    # jogar objeto no chão
                    game_state['env manager'].ambiente_atual().joga_objeto_no_chão(removed)

                print()
                print(obj.nome,'->', obj.mensagem)
                time.sleep(5)
                
                print(game_state['inventário'])
                
            elif action == 'drop':
                
                # obtém o nome do objeto a descartar
                to_remove = cmd.replace('drop', '').strip()
                
                # tenta remover objeto 
                obj = game_state['inventário'].remove_objeto_do_inventário(to_remove)
                if obj:
                
                    # jogar objeto no chão
                    game_state['env manager'].ambiente_atual().joga_objeto_no_chão(obj)
                  
                else:
                    raise Exception("Não foi possível encontrar objeto")
              
            elif action == 'unstack':
                
                # obtém o índice da pilha a ser desempilhada
                idx_pilha = int(re.search(r'\d', cmd).group(0))
                
                # obtém objeto da pilha desejada
                m2 = game_state['env manager'].ambiente_atual().chão.metros_quadrados[idx_pilha]
                obj = m2.desempilha_objeto()
                
                # dropa objeto aleatoriamente em um dos outros metros quadrados
                game_state['env manager'].ambiente_atual().joga_objeto_no_chão(obj, ignore=[idx_pilha])
                
            elif action == 'next_env':
                
                # avança para próximo ambiente e salva ambiente atual
                game_state['env manager'].next()

            elif action == 'prev_env':
              
                # retorna para ambiente anterior e salva ambiente atual
                game_state['env manager'].previous()

            elif action == 'interact':

                # obtém nome do móvel (objeto interativo) que jogador deseja interagir
                nome_móvel = cmd.replace('interact', '').replace('with', '').strip()
                móvel = [i for i in game_state['env manager'].ambiente_atual().móveis if re.match(i.nome, nome_móvel, re.IGNORECASE)]

                # verifica se móvel existe no ambiente
                if not móvel:
                    raise Exception("%s não existe no ambiente atual" % nome_móvel)

                móvel = móvel[0]

                print()
                print(móvel.mensagem)
                print("\nAjuda:")
                móvel.help()
                print()

                while 1:

                    interação = input("Insira uma interação com o %s: " % móvel.nome).strip()

                    try:

                        if móvel.nome == "Janela":
                            móvel.allow_break = 'Pedra' in [i.nome for i in inventário.inventário]

                        method_to_call = getattr(móvel, interação)
                        method_response = method_to_call()
                        
                        if móvel.nome == "Janela" and method_response == "CRASH!":
                            print("\nParabéns, você conseguiu escapar!!")
                            time.sleep(2)
                            print("\nVocê conseguiu coletar os seguintes itens:")
                            for i in inventário.inventário:
                                print(i.nome)
                            win = True
                            break

                    except Exception as exception:
                        raise exception

        except Exception as exception:
            print("Comando inválido")
            print(exception)
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, "Error on line %s" % exc_tb.tb_lineno)
            continue
    

    game_end = dt.now()

    time.sleep(2)

    print("\n%s, você completou o jogo em %s" % (player_name, str(game_end - game_start))) 

